# Лабораторна робота 3 з "Асиметричних криптосистем та протоколів"
## Тема: Криптосистема Рабіна. Атака на протокол доведення без розголошення.

**Виконали**\
Бондар Петро, ФІ-03\
Кістаєв Матвій, ФІ-03

In [43]:
import hashlib
import random as rnd
import requests
from optimus import *

In [44]:
KEY_LENGTH = 256


class Rabin_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/rabin/'
    s = requests.Session()
    n = None
    b = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self, key_l: int) -> (str, str):
        req = f'{self.__base_url}serverKey?keySize={key_l}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n, self.b = (int(r['modulus'], 16), int(r['publicExponent'], 16))
        return (self.n, self.b)
    
    # Ask server to encrypt
    def encrypt(self, M: str, rec_n, rec_b, type='TEXT'):
        req = f'{self.__base_url}encrypt?modulus={format(rec_n, "X")}&b={format(rec_b, "X")}&message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return (r['cipherText'], r['parity'], r['jacobiSymbol'])
    
    # Ask server to decrypt this message with his private keys
    def decrypt(self, C: str, p: int, j: int, type='TEXT'):
        req = f'{self.__base_url}decrypt?cipherText={C}&expectedType={type}&parity={p}&jacobiSymbol={j}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['message']
    
    # Ask server to sign this message with his private keys
    def sign(self, M: str, type='TEXT'):
        req = f'{self.__base_url}sign?message={M}&type={type}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['signature']
    
    # Verify the message using this public key
    def verify(self, M: str, sign: str, u_n, type='TEXT'):
        req = f'{self.__base_url}verify?message={M}&type={type}&signature={sign}&modulus={format(u_n, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['verified']


In [45]:
def str2hex(s: str):
    res = ""

    for c in s:
        cb = hex(ord(c))
        res += cb[2::]

    return res

def num2str(n: int):
    text = str()
    while n != 0:
        text += chr(n % 256)
        n //= 256
    
    return text[::-1]

def rabin_format(X: int):
    return 255*(2**(KEY_LENGTH*2 - 16)) + int(str2hex(X), 16) * (2**64) + random.randrange(0, 2**64)

def rabin_unformat(X: int):
    return (X % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)

In [46]:
class Rabin_User:
    def __init__(self, p, q, b = 0, serv = Server()):
        print("Initializing user...")
        if not check_prime(p) or not check_prime(q):
            raise RuntimeError("p or q is not a prime number.")
        
        self.serv = serv
        self.p = p
        self.q = q
        self.b = b
        self.n = p*q
        self.get_server_public_key(KEY_LENGTH * 2)

        print(f"User private key (p, q): {(self.d, self.p, self.q)}")
        print(f"User public key (n): {(self.n, self.e)}")
        print(f"User server public key (n): {(self.serv.n, self.serv.e)}")
        print("--------------------------------------------------------")
    

    def get_server_public_key(self, len: int):
        self.serv.set_server_key(len)


    def send_message(self, M: str):
        print("Sending message to the server...")

        M_format = rabin_format(int(str2hex(M), 16))
        
        b1 = M_format % 2
        b2 = jacobi(M_format, self.serv.n)

        C = pow(M_format, 2, self.serv.n)

        M1 = self.serv.decrypt(C, b1, b2)
        
        check = (M1 == M)

        print(f"Sent message: {M}")
        print(f"Sent cyphertext: {C}")
        print(f"Server responce: {M1}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def send_message_sign(self, M: str):
        print("Sending signature to the server...")
        
        S = format(pow(int(str2hex(M), 16), self.d, self.n), 'X')

        check = self.serv.verify(M, S, self.n, self.e)

        print(f"Sent message: {M}")
        print(f"Sent signature: {S}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")


    def receive_message(self, M: str):
        print("Sending request for message to the server...")
        
        C, b1, b2 = self.serv.encrypt(M, self.n, self.b)
        Roots = sqrt_modpq(int(C, 16), self.p, self.q)

        for m in Roots:
            if [b1, b2] == [m % 2, (jacobi(m, self.p*self.q) == 1)]:
                M1 = m
                break
        
        M2 = (M1 % (pow(2, (KEY_LENGTH*2 - 16)))) // (2**64)
        M2 = num2str(M2)

        check = (M == M2)

        print(f"Sent message: {M}")
        print(f"Received cyphertext: {C}")
        print(f"Decoded cyphertext: {M2}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
    def receive_message_sign(self, M: str):
        print("Sending request for message signature to the server...")
        
        S = self.serv.sign(M)
        M1 = num2str(pow(int(S, 16), self.serv.e, self.serv.n))
        
        check = (M1 == M)

        print(f"Sent message: {M}")
        print(f"Received signature: {S}")
        print(f"Message signed with signature: {M1}")
        
        if check:
            print("Success")
        else:
            print("Error")
        print("--------------------------------------------------------")
        
        
        

SyntaxError: cannot assign to expression here. Maybe you meant '==' instead of '='? (859301257.py, line 61)

In [47]:
class ZNP_Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/znp/'
    s = requests.Session()
    n = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self) -> (str, str):
        req = f'{self.__base_url}serverKey'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        self.n = int(r['modulus'], 16)
        return self.n
    
    def take_sqrt_mod_n(self, y):
        req = f'{self.__base_url}challenge?y={format(y, "X")}'
        print(f"H: Sent request: {req}")
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code}:\n\tRequest {req}\n\tResponse{r.json()}")
        r = r.json()
        print(f"S: Response: {r}")
        return r['root']


class ZNP_User:
    def __init__(self, serv = ZNP_Server()):
        print("Initializing user...")
        
        self.serv = serv
        self.get_server_public_key()

        print(f"User server public key (n): {self.serv.n}")
        print("--------------------------------------------------------")

    def get_server_public_key(self):
        self.serv.set_server_key()

    # Ask server to 
    def attack_server(self):
        print("Starting ZNP attack...")

        print(f"Server public key: {self.serv.n}")

        itr = 0
        while True:
            x1 = rnd.randint(1, self.serv.n)
            itr += 1

            if jacobi(x1, self.serv.n) != -1:
                continue
            print(f"Randomed {itr} times")

            print("\n==== Asking to take root =====")
            x2 = int(self.serv.take_sqrt_mod_n(pow(x1, 2, self.serv.n)), 16)
            print("========================")
            print(f"\nCandidate: {x2}")
            
            if x1 != x2 and x1 != ((-x2) % self.serv.n):
                print("Candidate is OK!")
                break

        p = math.gcd(x1 - x2, self.serv.n)
        if p == 1:
            p = math.gcd(x1 + x2, self.serv.n)
        q = self.serv.n // p

        print("\n==Results==")
        print(f"p: {p}")
        print(f"q: {q}")
        print(f"p*q: {p*q}")
        print("=========\n")

        check = (p*q == self.serv.n)
        if check:
            print("Success!")
        else:
            print("No Success. ЩЗХ? :(")
        
        print("--------------------------------------------------------")

In [48]:
znp_u = ZNP_User()

znp_u.attack_server()

Initializing user...
H: Sent request: http://asymcryptwebservice.appspot.com/znp/serverKey
S: Response: {'modulus': '9E1ADEB70527E00F8AA344BEF0C15DCF70CDD10CEBDB8E10CC80ED45DFCBAD62BFE86D7F63259898EBFEAD3719AEFB3E204D4FC26E71C2B36ABE572720F1E29B90AA36DBA3EE5AC9F805C4B9A624DB5E0D6070682E7261C6350037095B098CC8E83486E68FEB57A41AC77F62E952CD763BE3B77DE1187B7FF0699E9ECE3791129892BA45D261A1314C404BB208C15EF17AAF43A40842DFAC11A4CFC9DCE716E81D48ED483AE41EB49BCE2564890A8AC076A7035024F307C103BDDC0D2D981EA8906D19684326F548F6240E1C704FFAC048020046F6DAF758536B1B65CE3830C69C2ED12990E4F6CE18EA798EFA90969AE854D17BF36670EC3196ED23F50B9DC9'}
User server public key (n): 1995890226569425575604795599216132791167287302453967022224271397013930401154173499254879106392239652991658299902027908475323639318603837210350689145105805419380537502204674954956573080873982557223793755522979076579380377217344840061138006903011339324398140395365651699005631833115295152157549217760636467584771683332339502660158785576183711

True